In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

from sklearn.cluster import DBSCAN

particle2pdg={11:'eminus',-11:'eminus',13:'muon',-13:'muon',22:'gamma',211:'piminus',-211:'piminus',2212:'proton'}
pdg2instance={'eminus':3,'gamma':4,'muon':6,'piminus':8,'proton':9}

from larcv import larcv

from MCNN_uboone import UbooneConfig
class ShapesConfig(UbooneConfig):
#     RPN_ANCHOR_RATIOS = [0.25, 0.5,  1,  2, 4]
    RPN_ANCHOR_RATIOS = [0.5,  1,  2]
config = ShapesConfig()
config.display()

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def getRGBfromI(RGBint):
#     print RGBint
    RGBint=int(RGBint)
    blue =  RGBint & 255
    green = (RGBint >> 8) & 255
    red =   (RGBint >> 16) & 255
    return red, green, blue

from MCNN_uboone import UbooneDataset
dataset_train = UbooneDataset("/scratch/ruian/training_data/no_pion.root")
# dataset_train = UbooneDataset("/scratch/ruian/training_data/track_par_val.root")
# dataset_train = UbooneDataset("/scratch/ruian/training_data/75_200_mul_2.root")


# dataset_val = UbooneDataset("/data/dayajun/toymodel/uboone/train_data/75_200.root")
# dataset_val.load_events(1, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
# dataset_val.prepare()

In [ ]:
dataset_train.load_events(100, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()
print len(dataset_train.image_info)

In [ ]:
print dataset_train.class_info

In [ ]:
# reload(modellib)
# reload(visualize)
# reload(utils)
#Look at dataset_val
fig, (ax0,ax1) = plt.subplots(1,2,figsize=(20,10))
# fig, ax0 = plt.subplots(1,1,figsize=(10,10))
# fig, ax1 = plt.subplots(1,1,figsize=(10,10))

class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
inference_config = InferenceConfig()

image_id=0
# print 'image_id ',image_id
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, inference_config, 
                           image_id, use_mini_mask=False)

# print np.sum(original_image)
    
# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", str(gt_class_id))
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)
# print np.sum(original_image)

#     print '===========Detection==========='
ax0.imshow(original_image[:,:,0], origin='lower')


# ax0.set_ylim(512 + 10, -10)
# ax0.set_xlim(-10, 512 + 10)


visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8),ax=ax1)
ax1.text(10,450,'image_id: %i'%image_id, color='w', fontsize=25)

# print gt_bbox
# print gt_class_id
# print dataset_train.class_names 
boxes=dataset_train.load_bbox(image_id)

# print gt_bbox
print set(gt_mask.flatten())
print gt_mask.shape

In [ ]:
table=dataset_train.class_info
x=40
for x in range(x,x+20):
    fig, (ax0,ax1) = plt.subplots(1,2,figsize=(20,10))
    image_id=x
    # print 'image_id ',image_id
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_train, inference_config, 
                               image_id, use_mini_mask=False)

#     print gt_bbox
#     print gt_class_id
#     print dataset_train.class_names 
    
    # print np.sum(original_imag
    ax0.imshow(original_image[:,:,0], origin='lower')
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8),ax=ax1)
#     ax1.text(10,450,'image_id: %i'%image_id, color='w', fontsize=25)
#     for i in xrange(len(gt_class_id)):
#         ax1.text(10,410-i*30,'%ith pdg: %i'%(i,table[gt_class_id[i]]['name']), color='w', fontsize=25)



In [ ]:
import numpy as np
a=b=np.array([1])
b[0]=2
print a

In [ ]:
b=np.array([1])
a=b.copy()
b[0]=2
print a

In [ ]:
test= np.array([1,2,3,4,5])
# test[test==1]=0
# test[test==2]=0
print (test==1) + (test==2)
print test

In [ ]:
test = dataset_train.load_bbox(0)
print test

In [ ]:
test[:,:,1:]

In [ ]:
test=np.zeros((2,4,2))
print test[0]
# test[0]=np.array([])

In [ ]:
# print dataset_train.image_ids
image_ids = np.random.choice(dataset_train.image_ids, 2)
# image_ids=[0,1]
# print image_ids
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
#     print 'image_id',image_id
#     print image.shape
#     print set(image.flatten())
#     print np.sum(image)
#     print mask.shape
#     for i in xrange(mask.shape[-1]):
#         print np.sum(mask[:,:,i]),
#     print ' '
    print class_ids
    visualize.display_top_masks(image[:,:,0], mask, class_ids, dataset_train.class_names)

# Training

In [ ]:
# # Create model in training mode
# model = modellib.MaskRCNN(mode="training", config=config,
#                           model_dir=MODEL_DIR)


In [ ]:
# # Training Heads
# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
#             epochs=1, 
#             layers='heads')

In [ ]:
# # Training ALl
# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE / 10,
#             epochs=2, 
#             layers="all")

# Detection

In [ ]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weight
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
# model_path = model.find_last()
model_path="/data/dayajun/sw/Mask_RCNN/logs/particles20181001T1620/mask_rcnn_particles_0200.h5"

# Load trained weights
print("Loading weights from ", model_path)

model.load_weights(model_path, by_name=True)

In [ ]:
from keras.utils import plot_model
# plot_model(model.keras_model)
print len(model.keras_model.layers)

In [ ]:
# reload(visualize)
# reload(utils)
#Look at dataset_val
fig, (ax0,ax1) = plt.subplots(1,2,figsize=(16,8))

image_id = random.choice(dataset_train.image_ids)
image_id=6
print 'image_id ',image_id
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, inference_config, 
                           image_id, use_mini_mask=False)

# print np.sum(original_image)
    
# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)
# print np.sum(original_image)

#     print '===========Detection==========='

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8),ax=ax0)
print np.sum(original_image)
results = model.detect([original_image], verbose=0)
print np.sum(original_image)

r = results[0]
#     print r['rois'].shape
#     print r['masks'].shape
#     print r['class_ids'].shape
#     print np.sum(original_image)

# print np.sum(r['masks'])

visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_train.class_names, r['scores'], ax=ax1)

# visualize.display_differences(original_image,gt_bbox, gt_class_id, gt_mask, r['rois'],\
#                               r['class_ids'],r['scores'],r['masks']\
#                              ,dataset_train.class_names)

In [ ]:
dataset = dataset_val
for image_id in xrange(0,50):
    fig, (ax0,ax1) = plt.subplots(1,2,figsize=(16,8))

    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, inference_config, 
                               image_id, use_mini_mask=False)

#     log("original_image", original_image)
#     log("image_meta", image_meta)
#     log("gt_class_id", gt_class_id)
#     log("gt_bbox", gt_bbox)
#     log("gt_mask", gt_mask)
#     print np.sum(original_image)

    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                                dataset.class_names, figsize=(8, 8),ax=ax0)

    results = model.detect([original_image], verbose=0)
    r = results[0]
            
    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=ax1)


In [ ]:
fig, ax= plt.subplots(1,1,figsize=(8,6))
ax.plot([0,0],[1,10],'-')

In [ ]:
test=np.zeros(shape=[512,512,5])
# print test.shape
for i in range(5):
    test[:,:,i].fill(i)
np.delete(test, 1)
new=np.zeros([512,512])
new.fill(6)
# print test[:,:]
# print new

shit=np.dstack((test, new))

# print shit

print shit.shape
for i in range(6):
    print '.....................'
    print  shit[:,:,i]

In [ ]:
test=np.zeros(shape=[512,512,5])
# print test.shape
for i in range(5):
    test[:,:,i].fill(i)
np.delete(test, 1)
new=np.zeros([512,512,1])
new[:,:,0].fill(6)
# print test[:,:]
# print new

print test.shape
print new.shape

shit=np.concatenate((test, new),axis=2)

# print shit

print shit.shape
for i in range(6):
    print '.....................'
    print  shit[:,:,i]

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5# Comput 
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
test=np.array([[1,2,3],[1,2,3]])
cover=np.array([[1,1,1],[1,1,0]])
print test.shape
print cover.shape

shit=test*cover
print shit.shape

In [ ]:
print shit

In [ ]:
np.zeros((2,1,0))